In [16]:
import pandas as pd
import numpy as np
import math
import copy
import QSTK.qstkutil.qsdateutil as du
import datetime as dt
import QSTK.qstkutil.DataAccess as da
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkstudy.EventProfiler as ep
import csv
import datetime as dt
import matplotlib.pyplot  as plt




#第二周作业
def preEventData():
    dataobj = da.DataAccess('Yahoo')
    ls_symbols = dataobj.get_symbols_from_list("sp5002012")
    ls_symbols.append('SPY')
    dt_start = dt.datetime(2008, 1, 1)
    dt_end = dt.datetime(2009, 12, 31) 
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess('Yahoo')
    ls_keys = ['actual_close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)   
    d_data = dict(zip(ls_keys, ldf_data))
    for s_key in ls_keys:
        d_data[s_key] = d_data[s_key].fillna(method = 'ffill')
        d_data[s_key] = d_data[s_key].fillna(method = 'bfill')
        d_data[s_key] = d_data[s_key].fillna(1.0)
    return d_data,ls_symbols

def eventStudy():
    d_data,ls_symbols = preEventData()#pd.read_csv('sp5002008.csv',parse_dates=True)

    df_close = d_data['actual_close']
    ldt_timestamps = df_close.index 
    
    print 'compute event'
    
    df_event = []
    for s_sym in ls_symbols:
        for i in range(1,len(ldt_timestamps)):
            f_symprice_today = df_close[s_sym].ix[ldt_timestamps[i]]
            f_symprice_yest = df_close[s_sym].ix[ldt_timestamps[i - 1]]
            if f_symprice_today < 5.0 and f_symprice_yest >= 5.0:
                df_event.append([ldt_timestamps[i].year , ldt_timestamps[i].month , ldt_timestamps[i].day , s_sym , 'Buy' , 100])
                if(i + 5 < len(ldt_timestamps)):
                    sellDay = i + 5
                else:
                    sellDay = len(ldt_timestamps) - 2
                df_event.append([ldt_timestamps[sellDay].year , ldt_timestamps[sellDay].month , ldt_timestamps[sellDay].day , s_sym , 'Sell' , 100])
    
    with open('orders.csv', 'wb') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerows(df_event)
    return df_event   
    
    

#对组合进行评价
def polio_val(raw_data):
    daily_data = copy.deepcopy(raw_data);
    polio=[1]
    dailyRet=[0]  
    for i in range(1,len(raw_data),1):
        fixedDay = (daily_data[i])/daily_data[0]
        polio.append(fixedDay)
        dailyRet.append((polio[i] - polio[i-1])/polio[i-1])
    
    print 'Sharpe Ratio of Fund : ' +  str(math.sqrt(252) * np.array(dailyRet).mean() / np.array(dailyRet).std())
    print 'Total Return of Fund : ' +  str(polio[len(polio) - 1])   
    print 'Standard Deviation of Fund :  ' + str(np.array(dailyRet).std())
    print 'Average Daily Return of Fund : ' + str(np.array(dailyRet).mean())

#获取股票价格数据    
def price_matrix(dt_start, dt_end , ls_symbols):
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    c_dataobj = da.DataAccess('Yahoo')
    ls_keys = ['close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)   
    d_data = dict(zip(ls_keys, ldf_data))
    df_close = d_data['close']
    for s_key in ls_symbols:
        df_close[s_key] = df_close[s_key].fillna(method='ffill')
        df_close[s_key] = df_close[s_key].fillna(method='bfill')
    return df_close

#模拟市场数据 算出来盈利
def market_sim(filename='orders.csv' , cash=50000):
    dates=[]
    symbols=[]
    reader = csv.reader(open(filename , 'rU') , delimiter=',')
    for row in reader:
        dates.append(dt.datetime(int(row[0]), int(row[1]), int(row[2])))
        symbols.append(row[3])
    
    #remove duplicate
    dates = list(set(dates))
    dates.sort()
    #多增加一天
    dates.append(dates[len(dates)-1] + dt.timedelta(days=1))
    
    
    symbols = list(set(symbols))
    dt_start = dates[0]
    dt_end = dates[len(dates)-1]
    dt_timeofday = dt.timedelta(hours=16)
    #dates为交易日期
    dates = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
  
    #构建交易矩阵
    trade_matrix = pd.DataFrame(0 , index=dates , columns=symbols)
    trade_matrix['CASH'] = cash
    
    reader = csv.reader(open(filename , 'rU') , delimiter=',')
    
    for row in reader:
        dtime = dt.datetime(int(row[0]), int(row[1]), int(row[2])) + dt.timedelta(hours=16)
        if(row[4] == 'Sell'):       
            trade_matrix[str(row[3])].ix[dtime] = trade_matrix[str(row[3])].ix[dtime]-100
        elif(row[4] == 'Buy'):
            trade_matrix[str(row[3])].ix[dtime] = trade_matrix[str(row[3])].ix[dtime]+100
        else:
            print row[4]


        
            
            
    trade_matrix.to_csv('trade.csv')
    #仓位矩阵
    holds_matrix = copy.deepcopy(trade_matrix)
    
    
    for s_sym in symbols:
        for i in range(1, len(dates)):
            holds_matrix[s_sym].ix[dates[i]] = holds_matrix[s_sym].ix[dates[i-1]] + trade_matrix[s_sym].ix[dates[i]]   
    
    #求得前复权的收盘价格矩阵
    df_close = price_matrix(dt_start ,dt_end ,symbols)   
     
    for s_sym in symbols:
        holds_matrix['CASH'].ix[dates[0]] -= 1 * trade_matrix[s_sym].ix[dates[0]] * df_close[s_sym].ix[dates[0]]
  
    for i in range(1, len(dates)):
        last_cash = holds_matrix['CASH'].ix[dates[i-1]]
        trade_sum = 0
        for s_sym in symbols:
            trade_sum += 1 * trade_matrix[s_sym].ix[dates[i]] * df_close[s_sym].ix[dates[i]]
        holds_matrix['CASH'].ix[dates[i]] = last_cash - trade_sum       
    
    df_close['CASH'] = 1.0


    df_val =df_close * holds_matrix
    for s_sym in symbols:
        df_val['CASH']+=df_val[s_sym]
        
    
    return df_val['CASH'],dt_start ,dt_end 


#回测
def back_testing():
    df_val,dt_start ,dt_end  = market_sim()
    df_val.to_csv('data/marketsim.csv')
    df_standard = price_matrix(dt_start ,dt_end ,['$SPX'])
    print 'Data Range :  '+str(dt_start)+' to  '+str(dt_end)
    #计算此组合的各项指标
    polio_val(df_val)
    #计算对比组合的各项指标
    print '\n'
    polio_val(df_standard['$SPX'])

def bollinger_bands(symbols , dt_start , dt_end):
    df_close = price_matrix(dt_start , dt_end , symbols)
    rolling_mean = pd.rolling_mean(df_close , 20)
    rolling_std = pd.rolling_std(df_close , 20)
    bollinger_val = (df_close - rolling_mean) / (rolling_std)
    
    for sym in symbols:
        plt.figure()
        p1 = plt.subplot(211)
        p2 = plt.subplot(212)
        p1.plot(df_close.index , df_close[sym] , 'k')
        p1.plot(rolling_mean.index , rolling_mean[sym] , 'b')
        p1.fill_between(rolling_std.index , rolling_mean[sym] - 2 * rolling_std[sym] , rolling_mean[sym] + 2 * rolling_std[sym],
                    color='b' , alpha=0.2)
        
        p2.plot(bollinger_val.index , bollinger_val[sym] , 'b')
        p2.fill_between(bollinger_val.index , -1 , 1, color='b' , alpha=0.2)
        
        plt.savefig(sym+'.pdf')
    return bollinger_val


    
if __name__ == '__main__':
    #此处进行event 研究，产生orders.csv文件
    #eventStudy()
    #此处根据event研究的结果进行回测模拟
    #back_testing()
    bollinger_bands(['AAPL','GOOG','IBM','MSFT'] , dt.datetime(2010,1,1) , dt.datetime(2010,12,31))
    
    
   

